## R-alpha

In [1]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp
!pip install requests
!pip install numpy
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream, token_lookup, get_instrument_list, hist_data, hist_data_from_now
from strategy import expected_buy_price, expected_sell_price, trend_detection, exit_long, exit_short
from order import place_limit_order, place_market_order
import json
import threading
import time   

In [2]:
api_key = '2VTNvx7z'
username = 'R258447'
pwd = '1499'
token_id = '3URP5C2ZUQSYTSEFELMM2CFJGM'

In [3]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

[I 240504 15:33:54 smartConnect:121] in pool


### Order Book Strategy

In [4]:
instrument_list = get_instrument_list()

ticker_list = ["ITC"]
token_list_0 = []

for ticker in ticker_list:
    token = token_lookup(ticker, instrument_list, exchange="NSE")
    token_list_0.append(token)

token_list = [
        {
            "exchangeType": 1,
            "tokens": token_list_0
        }
    ]

bid_ask_ratio = {}
long_trades = {}
short_trades = {}

for ticker in ticker_list:
    bid_ask_ratio[ticker] = []
    long_trades[ticker] = []
    short_trades[ticker] = []

pos_size = 10000

long_pos = []
short_pos = []

print(token_list)

[{'exchangeType': 1, 'tokens': ['1660']}]


In [18]:
correlation_id = "stream_2" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 3 #1 for LTP, 2 for Quote and 3 for SnapQuote
    
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

In [19]:
def token_lookup(ticker, instrument_list, exchange="NSE"):
    for instrument in instrument_list:
        if instrument["name"] == ticker and instrument["exch_seg"] == exchange and instrument["symbol"].split('-')[-1] == "EQ":
            return instrument["token"]
        
def symbol_lookup(token,exchange="NSE"):
    for instrument in instrument_list:
        if instrument["token"] == token and instrument["exch_seg"] == exchange:
            return instrument["symbol"][:-3]

def stream_list(list_stocks,exchange="nse_cm"):
    #SAMPLE: nse_cm|2885&nse_cm|1594&nse_cm|11536&nse_cm|3045
    # token="mcx_fo|226745&mcx_fo|220822&mcx_fo|227182&mcx_fo|221599"
    return_string = ''
    for count,ticker in enumerate(list_stocks):
        if count != 0:
            return_string+= '&'+exchange+'|'+token_lookup(ticker,instrument_list)
        else:
            return_string+= exchange+'|'+token_lookup(ticker,instrument_list)
    return return_string

def expected_buy_price(tick, pos_size):
    ask_prices = [float(tick[i]) for i in tick if i[:2]=="sp"]
    ask_sizes = [float(tick[i]) for i in tick if i[:2]=="bs"]
   
    cum_pos = 0
    cum_size = 0
    prev_fill = 0
    prev_price = 0
    
    for i in range(len(ask_prices)):
        cum_pos+= ask_prices[i]*ask_sizes[i]
        cum_size+= ask_sizes[i]
        if pos_size <= cum_pos:
            return ((prev_price*prev_fill) + (ask_prices[i]*(pos_size - prev_fill)))/pos_size
        else:
            prev_fill = cum_pos
            prev_price = round(cum_pos/cum_size,2)
    return round(cum_pos/cum_size,2)
        
def expected_sell_price(tick, pos_size):
    bid_prices = [float(tick[i]) for i in tick if i[:2]=="bp"]
    bid_sizes = [float(tick[i]) for i in tick if i[:2]=="bq"]
    
    cum_pos = 0
    cum_size = 0
    prev_fill = 0
    prev_price = 0
    
    for i in range(len(bid_prices)):
        cum_pos+= bid_prices[i]*bid_sizes[i]
        cum_size+= bid_sizes[i]
        if pos_size <= cum_pos:
            return ((prev_price*prev_fill) + (bid_prices[i]*(pos_size - prev_fill)))/pos_size
        else:
            prev_fill = cum_pos
            prev_price = round(cum_pos/cum_size,2)
    return round(cum_pos/cum_size,2)
    
            
def trend_detection(ratio_list):
    if len(ratio_list) >= 30 and max(ratio_list) >= 10 and min(ratio_list) >= 5 and mean(ratio_list) >= 8:
        return "Buy"
    elif len(ratio_list) >= 30 and min(ratio_list) <= 0.1 and max(ratio_list) <= 0.2 and mean(ratio_list) <= 0.125 :
        return "Sell"
    else:
        return 0
    
def exit_long(ratio_list):
    if max(ratio_list) <= 4 and min(ratio_list) <= 1.5 and mean(ratio_list) <= 2:
        return True
    else:
        return False
    
def exit_short(ratio_list):
    if min(ratio_list) >= 0.25 and max(ratio_list) >= 0.65 and mean(ratio_list) >= 0.4:
        return True
    else:
        return False
    
def market_pressure(tick):
    token = tick['token']
    tot_bid_vol = 0
    tot_ask_vol = 0
    buy_data = tick['best_5_buy_data']
    sell_data = tick['best_5_sell_data']
    for bd in buy_data:
        tot_bid_vol+= int(bd['quantity'])
    for sd in sell_data:
        tot_ask_vol+= int(sd['quantity'])

    bid_ask_ratio[symbol_lookup(token)].append(tot_bid_vol/tot_ask_vol)
    if len(bid_ask_ratio[symbol_lookup(token)]) > 30:
        bid_ask_ratio[symbol_lookup(token)].pop(0)
    
    if symbol_lookup(token) not in long_pos:
        if trend_detection(bid_ask_ratio[symbol_lookup(token)]) == "Buy":
            print("{}: buy {} at price {} : total bid volume = {}, total ask volume = {}".format(dt.datetime.now(),symbol_lookup(token),expected_buy_price(tick,pos_size),tot_bid_vol,tot_ask_vol))
            long_pos.append(symbol_lookup(token))
            long_trades[symbol_lookup(token)].append([expected_buy_price(tick,pos_size)])
    else:
        if exit_long(bid_ask_ratio[symbol_lookup(token)]):
            print("{}: close long {} at price {} : total bid volume = {}, total ask volume = {}".format(dt.datetime.now(),symbol_lookup(token),expected_sell_price(tick,pos_size),tot_bid_vol,tot_ask_vol))
            long_pos.remove(symbol_lookup(token))
            long_trades[symbol_lookup(token)][-1].append(expected_sell_price(tick,pos_size))
    if symbol_lookup(token) not in short_pos:
        if trend_detection(bid_ask_ratio[symbol_lookup(token)]) == "Sell":    
            print("{}: sell {} at price {}: total bid volume = {}, total ask volume = {}".format(dt.datetime.now(),symbol_lookup(token),expected_sell_price(tick,pos_size),tot_bid_vol,tot_ask_vol))
            short_pos.append(symbol_lookup(token))
            short_trades[symbol_lookup(token)].append([expected_sell_price(tick,pos_size)])
    else:
        if exit_short(bid_ask_ratio[symbol_lookup(token)]):
            print("{}: close short {} at price {} : total bid volume = {}, total ask volume = {}".format(dt.datetime.now(),symbol_lookup(token),expected_buy_price(tick,pos_size),tot_bid_vol,tot_ask_vol))
            short_pos.remove(symbol_lookup(token))
            short_trades[symbol_lookup(token)][-1].append(expected_buy_price(tick,pos_size))

    print(symbol_lookup(token), "last 5 bid ask ratios :", bid_ask_ratio[symbol_lookup(token)][-5:])

def on_data(ws, message):
    market_pressure(message)

sws.on_data = on_data

In [20]:
sws.connect()

[I 240426 06:53:24 data:60] on open


ITC last 5 bid ask ratios : [0.05102912196190059]
ITC last 5 bid ask ratios : [0.05102912196190059, 0.05102912196190059]
ITC last 5 bid ask ratios : [0.05102912196190059, 0.05102912196190059, 0.04795095125159232]
ITC last 5 bid ask ratios : [0.05102912196190059, 0.05102912196190059, 0.04795095125159232, 0.04795095125159232]
ITC last 5 bid ask ratios : [0.05102912196190059, 0.05102912196190059, 0.04795095125159232, 0.04795095125159232, 0.20592709098622738]
ITC last 5 bid ask ratios : [0.05102912196190059, 0.04795095125159232, 0.04795095125159232, 0.20592709098622738, 0.20592709098622738]
ITC last 5 bid ask ratios : [0.04795095125159232, 0.04795095125159232, 0.20592709098622738, 0.20592709098622738, 0.1466447191336936]
ITC last 5 bid ask ratios : [0.04795095125159232, 0.20592709098622738, 0.20592709098622738, 0.1466447191336936, 0.1466447191336936]
ITC last 5 bid ask ratios : [0.20592709098622738, 0.20592709098622738, 0.1466447191336936, 0.1466447191336936, 0.1486394064371723]
ITC last 5

[W 240426 06:53:44 smartWebSocketV2:319] Attempting to resubscribe/reconnect (Attempt 1)...
[I 240426 06:53:55 data:60] on open


ITC last 5 bid ask ratios : [6.973198847262248, 0.2432795992913231, 0.2432795992913231, 0.23426994993358538, 0.24484191046186557]
ITC last 5 bid ask ratios : [0.2432795992913231, 0.2432795992913231, 0.23426994993358538, 0.24484191046186557, 0.24484191046186557]
ITC last 5 bid ask ratios : [0.2432795992913231, 0.23426994993358538, 0.24484191046186557, 0.24484191046186557, 1.2167753870119755]
ITC last 5 bid ask ratios : [0.23426994993358538, 0.24484191046186557, 0.24484191046186557, 1.2167753870119755, 1.208427922814983]
ITC last 5 bid ask ratios : [0.24484191046186557, 0.24484191046186557, 1.2167753870119755, 1.208427922814983, 1.208427922814983]


## Momentum Strategy

In [ ]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# Alpha Vantage API parameters
API_KEY = '5PTSVQOUZRRRZN7S'
symbol = 'ITC'
interval = '5min'  # 5-minute interval data
outputsize = 'full'  # Full intraday data

# Initialize variables
pos1 = int(0)  # 1 for long position, -1 for short position, 0 for no position
capital = 1000  # Initial capital in USD
current_position_value = 0
last_price = 0

# Define parameters
lookback_period = 12  # Number of intervals to look back for momentum calculation
buy_threshold = 0.0000003  # Buy if the price increases by more than 0.5% in the lookback period
sell_threshold = -0.005  # Sell if the price decreases by more than 0.5% in the lookback period

df = pd.DataFrame(columns=['Price','Returns'])

# Fetch intraday price data from Alpha Vantage

def momentum_strategy(tick):
    token = tick['token']
    ltp = tick['last_traded_price']/100
    df.loc[len(df),'Price'] = ltp
    print(ltp)
    # Calculate momentum
    df['Returns'] = df['Price'].pct_change()
    df['Momentum'] = df['Returns'].rolling(window=lookback_period).sum()

    # Implement momentum strategy
    
    price = df['Price'].iloc[-1]
    momentum = df['Momentum'].iloc[-1]
  
    
    if len(df) > lookback_period:
        if momentum > buy_threshold:
            print('hiiii')
            # if pos1 != 1:
            current_position_value = capital
            # pos1 = 1
            last_price = price
            print(f"Buying at {price}")

        elif momentum < sell_threshold:
            # if pos1 != -1:  # Enter short position
            current_position_value = -capital
            # pos1 = -1
            last_price = price
            print(f"Selling short at {price}")



def on_data(ws, message):
    momentum_strategy(message)

sws.on_data = on_data

## misc

In [ ]:
correlation_id = "stream_1"
action = 1
mode = 1
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

In [ ]:
# Create a new thread and pass the function to it
my_thread = threading.Thread(target=sws.connect())

# Start the thread
my_thread.start()
time.sleep(10)

[I 240426 06:36:27 data:60] on open
[I 240426 06:36:28 data:53] Ticks: {'subscription_mode': 3, 'exchange_type': 1, 'token': '1660', 'sequence_number': 8635963, 'exchange_timestamp': 1714113387000, 'last_traded_price': 43945, 'subscription_mode_val': 'SNAP_QUOTE', 'last_traded_quantity': 200, 'average_traded_price': 43933, 'volume_trade_for_the_day': 6197033, 'total_buy_quantity': 858405.0, 'total_sell_quantity': 2360352.0, 'open_price_of_the_day': 43925, 'high_price_of_the_day': 44330, 'low_price_of_the_day': 43640, 'closed_price': 43755, 'last_traded_timestamp': 1714113383, 'open_interest': 164616000, 'open_interest_change_percentage': -4651119324912936352, 'upper_circuit_limit': 48130, 'lower_circuit_limit': 39380, '52_week_high_price': 49970, '52_week_low_price': 39935, 'best_5_buy_data': [{'flag': 1, 'quantity': 5017, 'price': 43945, 'no of orders': 4}, {'flag': 1, 'quantity': 209, 'price': 43940, 'no of orders': 3}, {'flag': 1, 'quantity': 876, 'price': 43935, 'no of orders': 10}

In [ ]:
place_market_order(instrument_list,ticker="ITC",buy_sell="BUY",price=0,quantity=1,obj=smartApi, sl=0,sqof=0,exchange="NSE")

'240426000628220'

In [17]:
nifty_50 = hist_data_from_now(smartApi, "NIFTYBEES", 30, "ONE_MINUTE", instrument_list)

In [18]:
nifty_50.to_csv("niftybees_30day_1min.csv")